In [15]:
import pandas as pd
import numpy as np
from pgmpy.estimators import BDeuScore, K2Score, BicScore
from pgmpy.models import BayesianNetwork
import scipy.stats

import sklearn

In [2]:
df = pd.read_csv("patients.csv")

In [3]:
is_null_feature_df = df.isnull().any()
is_null_indexs = is_null_feature_df.loc[is_null_feature_df == True].index
print(df[is_null_indexs].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   DOD                     41 non-null     object 
 1   DOD_HOSP                38 non-null     object 
 2   DOD_SSN                 41 non-null     object 
 3   DEATHTIME               23 non-null     object 
 4   LANGUAGE                46 non-null     object 
 5   EDREGTIME               66 non-null     object 
 6   EDOUTTIME               66 non-null     object 
 7   SUBMIT_WARDID           33 non-null     float64
 8   SUBMIT_CAREUNIT         6 non-null      object 
 9   CURR_WARDID             33 non-null     float64
 10  CURR_CAREUNIT           33 non-null     object 
 11  CALLOUT_WARDID          33 non-null     float64
 12  CALLOUT_SERVICE         33 non-null     object 
 13  REQUEST_TELE            33 non-null     float64
 14  REQUEST_RESP            33 non-null     floa

In [4]:
df2 = df.dropna(axis=1, how='any')
df.columns

Index(['Unnamed: 0', 'SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP',
       'DOD_SSN', 'EXPIRE_FLAG', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 'SUBMIT_WARDID',
       'SUBMIT_CAREUNIT', 'CURR_WARDID', 'CURR_CAREUNIT', 'CALLOUT_WARDID',
       'CALLOUT_SERVICE', 'REQUEST_TELE', 'REQUEST_RESP', 'REQUEST_CDIFF',
       'REQUEST_MRSA', 'REQUEST_VRE', 'CALLOUT_STATUS', 'CALLOUT_OUTCOME',
       'DISCHARGE_WARDID', 'ACKNOWLEDGE_STATUS', 'CREATETIME', 'UPDATETIME',
       'ACKNOWLEDGETIME', 'OUTCOMETIME', 'FIRSTRESERVATIONTIME',
       'CURRENTRESERVATIONTIME', 'ICUSTAY_ID', 'DBSOURCE', 'FIRST_CAREUNIT',
       'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID', 'INTIME', 'OUTTIME',
       'LOS', 'TRANSFERTIME', 'PREV_SERVICE', 'CURR_SERVICE', '

In [5]:
from numpy import NaN

# df2.loc[:, ['SUBJECT_ID']] = NaN
df2

,Unnamed: 0,SUBJECT_ID,GENDER,DOB,EXPIRE_FLAG,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,...,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,TRANSFERTIME,CURR_SERVICE,EVENTTYPE
0,0,376,F,2068-01-22,0,197503,2140-05-16 15:28:00,2140-05-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,...,MICU,MICU,15,15,2140-05-17 01:41:29,2140-05-19 20:13:13,2.7720,2140-05-16 15:29:39,MED,admit
1,1,376,F,2068-01-22,0,197503,2140-05-16 15:28:00,2140-05-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,...,MICU,MICU,15,15,2140-05-17 01:41:29,2140-05-19 20:13:13,2.7720,2140-05-16 15:29:39,MED,discharge
2,2,376,F,2068-01-22,0,197503,2140-05-16 15:28:00,2140-05-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,...,MICU,MICU,15,15,2140-05-17 01:41:29,2140-05-19 20:13:13,2.7720,2140-05-16 15:29:39,MED,transfer
3,3,376,F,2068-01-22,0,197503,2140-05-16 15:28:00,2140-05-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,...,MICU,MICU,15,15,2140-05-17 01:41:29,2140-05-19 20:13:13,2.7720,2140-05-16 15:29:39,MED,transfer
4,4,376,F,2068-01-22,0,160376,2140-06-07 21:37:00,2140-06-12 16:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,...,TSICU,TSICU,33,33,2140-06-07 21:39:15,2140-06-09 19:44:46,1.9205,2140-06-07 21:39:15,SURG,admit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,64,22192,F,1828-07-20,1,184281,2132-03-22 14:32:00,2132-04-03 16:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,...,MICU,MICU,52,52,2132-03-22 15:31:53,2132-03-29 22:54:34,7.3074,2132-03-22 14:32:59,MED,transfer
65,65,22192,F,1828-07-20,1,184281,2132-03-22 14:32:00,2132-04-03 16:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,...,MICU,MICU,52,52,2132-03-22 15:31:53,2132-03-29 22:54:34,7.3074,2132-03-22 14:32:59,MED,transfer
66,66,22192,F,1828-07-20,1,184281,2132-03-22 14:32:00,2132-04-03 16:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,...,MICU,MICU,52,52,2132-03-22 15:31:53,2132-03-29 22:54:34,7.3074,2132-03-22 14:32:59,MED,transfer
67,67,22192,F,1828-07-20,1,142604,2132-04-08 03:52:00,2132-04-13 19:58:00,EMERGENCY,EMERGENCY ROOM ADMIT,...,MICU,MICU,50,50,2132-04-08 03:52:56,2132-04-13 21:53:55,5.7507,2132-04-08 03:52:56,MED,admit


In [6]:
from pgmpy.estimators import HillClimbSearch

hc = HillClimbSearch(df2)
best_model = hc.estimate()
print(best_model.edges())

  0%|          | 0/1000000 [00:00<?, ?it/s]

[('SUBJECT_ID', 'DOB'), ('SUBJECT_ID', 'LAST_CAREUNIT'), ('SUBJECT_ID', 'GENDER'), ('GENDER', 'DBSOURCE'), ('DOB', 'RELIGION'), ('DOB', 'ETHNICITY'), ('HADM_ID', 'TRANSFERTIME'), ('ADMITTIME', 'DISCHARGE_LOCATION'), ('DISCHTIME', 'ADMITTIME'), ('DISCHARGE_LOCATION', 'INSURANCE'), ('DISCHARGE_LOCATION', 'HOSPITAL_EXPIRE_FLAG'), ('INSURANCE', 'EXPIRE_FLAG'), ('ETHNICITY', 'ADMISSION_LOCATION'), ('DIAGNOSIS', 'INTIME'), ('DIAGNOSIS', 'OUTTIME'), ('DIAGNOSIS', 'LAST_WARDID'), ('ICUSTAY_ID', 'HADM_ID'), ('ICUSTAY_ID', 'DIAGNOSIS'), ('FIRST_CAREUNIT', 'LAST_CAREUNIT'), ('FIRST_WARDID', 'LOS'), ('FIRST_WARDID', 'FIRST_CAREUNIT'), ('LAST_WARDID', 'MARITAL_STATUS'), ('INTIME', 'SUBJECT_ID'), ('LOS', 'DISCHTIME'), ('LOS', 'ICUSTAY_ID'), ('TRANSFERTIME', 'CURR_SERVICE'), ('TRANSFERTIME', 'Unnamed: 0'), ('EVENTTYPE', 'Unnamed: 0')]


In [7]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from pgmpy.sampling import BayesianModelSampling

In [8]:
my_model = BayesianNetwork(best_model.edges())

In [9]:
my_model.fit(df2, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5
for cpd in my_model.get_cpds():
    print(cpd)

+-------------------+-----+-----------------------------+
| INTIME            | ... | INTIME(2161-03-14 20:40:24) |
+-------------------+-----+-----------------------------+
| SUBJECT_ID(376)   | ... | 0.005780346820809248        |
+-------------------+-----+-----------------------------+
| SUBJECT_ID(4838)  | ... | 0.005780346820809248        |
+-------------------+-----+-----------------------------+
| SUBJECT_ID(11945) | ... | 0.005780346820809248        |
+-------------------+-----+-----------------------------+
| SUBJECT_ID(22192) | ... | 0.005780346820809248        |
+-------------------+-----+-----------------------------+
| SUBJECT_ID(32443) | ... | 0.976878612716763           |
+-------------------+-----+-----------------------------+
+-----------------+-----------------+-----+----------------------+
| SUBJECT_ID      | SUBJECT_ID(376) | ... | SUBJECT_ID(32443)    |
+-----------------+-----------------+-----+----------------------+
| DOB(1828-07-20) | 0.025           | ... | 0

In [10]:
samples = BayesianModelSampling(my_model).forward_sample(size=int(1e5))

  0%|          | 0/28 [00:00<?, ?it/s]

In [11]:
samples.loc[(samples['SUBJECT_ID'] == 11945), :]['EVENTTYPE']

3         transfer
4            admit
6         transfer
9         transfer
12        transfer
           ...    
99974     transfer
99976     transfer
99979        admit
99991     transfer
99992    discharge
Name: EVENTTYPE, Length: 29754, dtype: object

In [13]:
df2.columns

Index(['Unnamed: 0', 'SUBJECT_ID', 'GENDER', 'DOB', 'EXPIRE_FLAG', 'HADM_ID',
       'ADMITTIME', 'DISCHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'RELIGION', 'MARITAL_STATUS',
       'ETHNICITY', 'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'ICUSTAY_ID', 'DBSOURCE', 'FIRST_CAREUNIT',
       'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID', 'INTIME', 'OUTTIME',
       'LOS', 'TRANSFERTIME', 'CURR_SERVICE', 'EVENTTYPE'],
      dtype='object')

In [14]:
for i in df2.columns:
    if i not in samples.columns:
        print(i)

ADMISSION_TYPE
HAS_CHARTEVENTS_DATA


In [16]:
KL = scipy.stats.entropy(samples, df2)

TypeError: can't multiply sequence by non-int of type 'float'